## Conference Agent

In this notebook we create the conference sub-agent that uses abstracts from ESMO [European Society for Medical Oncology](https://www.esmo.org/). Esmo is an international conference in oncology organised annually. You can download [ESMO Open] (https://www.esmoopen.com/) abstracts freely. This notebook uses abstracts from ESMO Open March edition. 

Ensure the latest version of boto3 is shown below
If not then run through setup_environment.ipynb in the 0-Notebook-environment/ folder

In [18]:
!pip freeze | grep boto3

boto3==1.38.14


In [ ]:
!pip install --upgrade -q -r requirements.txt

## Agent Creation
In this section we create the sub-agent

In [ ]:
import boto3
import os

sts_client = boto3.client('sts')
session = boto3.session.Session()

account_id = sts_client.get_caller_identity()["Account"]
region = session.region_name
account_id_suffix = account_id[:3]
agent_suffix = f"{region}-{account_id_suffix}"
s3_client = boto3.client('s3', region)
bedrock_client = boto3.client('bedrock-runtime', region)

agent_foundation_model = [
    'anthropic.claude-3-haiku-20240307-v1:0',
    'anthropic.claude-3-sonnet-20240229-v1:0',
    'anthropic.claude-3-5-sonnet-20240620-v1:0'
]

In [21]:
agent_name = 'test2-esmo-collaborator-agent'
agent_description = "Collaborator agent responsible for handlign questions about ESMO abstracts"
agent_instruction = """You are a medical research assistant AI specialized in summarizing internal and external evidence related to oncology. For conference research, make use of the knowledge base to retrieve relevant information. Always cite abstract titles. """ 

Instantiate agent with the desired configuration

In [35]:
import sys

sys.path.insert(0, ".")
sys.path.insert(1, "..")

from utils.bedrock_agent_helper import (
    AgentsForAmazonBedrock
)
from utils.knowledge_base_helper import (
    KnowledgeBasesForAmazonBedrock
)
agents = AgentsForAmazonBedrock()
kb = KnowledgeBasesForAmazonBedrock()

In [ ]:

kb_info = kb.get_kb(kb_id)
kb_arn = kb_info['knowledgeBase']['knowledgeBaseArn']
#kb_id = <ENTER KB ID HERE> if you cannot pass variable between notebooks.

In [27]:
kb_config = {
    'kb_id': kb_id,
    'kb_instruction': 'Access the knowledge base when users ask about ESMO abstracts or latest conference information.'
}

In [24]:
agents = AgentsForAmazonBedrock()

conference_agent = agents.create_agent(
    agent_name,
    agent_description,
    agent_instruction,
    agent_foundation_model,
    code_interpretation=False,
    verbose=False
)

conference_agent

('RZL7IHMD0G',
 'TSTALIASID',
 'arn:aws:bedrock:us-east-1:774305571746:agent-alias/RZL7IHMD0G/TSTALIASID')

Extract useful agent information

In [25]:
conference_agent_id = conference_agent[0]
conference_agent_arn = f"arn:aws:bedrock:{region}:{account_id}:agent/{conference_agent_id}"

conference_agent_id, conference_agent_arn

('RZL7IHMD0G', 'arn:aws:bedrock:us-east-1:774305571746:agent/RZL7IHMD0G')

## Associate Knowledge Base with Agent
Finally, we can associate the new knowledge base with the agent using the AssociateAgentKnowledgeBase API from boto3

In [29]:
agents.associate_kb_with_agent(
    conference_agent[0], kb_config['kb_instruction'], kb_config['kb_id']
)

## Test your agent

In [39]:
import time
time.sleep(30)
bedrock_agent_runtime_client = boto3.client('bedrock-agent-runtime')

response = bedrock_agent_runtime_client.retrieve_and_generate(
    input={
        "text": "What were the key biomarkers assessed in gastric cancer samples in the LEGACY project?"
    },
    retrieveAndGenerateConfiguration={
        "type": "KNOWLEDGE_BASE",
        "knowledgeBaseConfiguration": {
            'knowledgeBaseId': kb_id,
            "modelArn": "arn:aws:bedrock:{}::foundation-model/{}".format(region, agent_foundation_model[2]),
            "retrievalConfiguration": {
                "vectorSearchConfiguration": {
                    "numberOfResults":1
                } 
            }
        }
    }
)

print(response['output']['text'],end='\n'*2)

The LEGACY project aimed to assess molecular hallmarks of gastric cancer using validated and emerging biomarkers. However, the specific key biomarkers are not explicitly mentioned in the given search result. The study involved an integrative analysis of clinical, epidemiological, and multi-omics data from geographically diverse samples, comparing European and Latin American cohorts. The project included immunohistochemistry (IHC) and in situ hybridization (ISH) assays, which are commonly used techniques for biomarker analysis, but the exact biomarkers assessed are not specified in the provided information.

